In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pi
import sys
sys.path.insert(1, 'scripts/')
from gen_matrix import matrix_gen, get_ICA
from get_sample import get_sample, create_strings_for_dataset
from fft import fft_for_sample
from tqdm import tqdm
import collections
%matplotlib inline

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import operator

import mne

import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_fnames = ['data/S001R06.edf',
              'data/S001R10.edf',
              'data/S001R14.edf']

raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])

eegbci.standardize(raw)  # set channel names
montage = make_standard_montage('standard_1005')
raw.set_montage(montage)

# strip channel names of "." characters
raw.rename_channels(lambda x: x.strip('.'))

eegbci.standardize(raw)  # set channel names
montage = make_standard_montage('standard_1005')
raw.set_montage(montage)

# strip channel names of "." characters
raw.rename_channels(lambda x: x.strip('.'))

ch_name_dict = {}

for i in range(0, 64):
    ch_name_dict[i] = raw.ch_names[i]

name_ch_dict = {}
for idx, i in enumerate(raw.ch_names):
    name_ch_dict[i] = idx

Extracting EDF parameters from /Users/alexandr/Documents/Git/MIPT_Masters_work/data/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/alexandr/Documents/Git/MIPT_Masters_work/data/S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/alexandr/Documents/Git/MIPT_Masters_work/data/S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


In [3]:
raw_fnames = ['data/S001R06.edf',
              'data/S001R10.edf',
              'data/S001R14.edf']

In [4]:
def get_class_eeg(raw_name_path, name):
    raw0 = read_raw_edf(raw_name_path, preload=True)
    eegbci.standardize(raw0)
    montage = make_standard_montage('standard_1005')
    raw0.set_montage(montage)
    raw0.rename_channels(lambda x: x.strip('.'))
    raw0.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')
    
    eeg1 = raw0.get_data()
    
    with open(f'data/S001/{name}_onsets_start.txt', 'r') as fin:
        start = list(map(lambda x: float(x.strip('\n')), fin.readlines()))
    
    with open(f'data/S001/{name}_onsets_end.txt', 'r') as fin:
        end = list(map(lambda x: float(x.strip('\n')), fin.readlines()))

    with open(f'data/S001/{name}_labels.txt', 'r') as fin:
        labels = list(map(lambda x: int(x.strip('\n')), fin.readlines()))
        
    one_colums_sec = end[-1] / eeg1.shape[1]
    
    eeg1_0 = np.zeros((64, 20000))
    eeg1_1 = np.zeros((64, 20000))
    eeg1_2 = np.zeros((64, 20000))

    for s,e,l in zip(start, end, labels):
        columns_start = round(s / one_colums_sec)
        columns_end = round(e / one_colums_sec)
        
        if columns_start == 1:
            columns_start -= 1

        if l == 0:
            eeg1_0[:, columns_start:columns_end] = eeg1[:, columns_start:columns_end]
        elif l == 1:
            eeg1_1[:, columns_start:columns_end] = eeg1[:, columns_start:columns_end]
        elif l == 2:
            eeg1_2[:, columns_start:columns_end] = eeg1[:, columns_start:columns_end]

            
    eeg1_0  = pd.DataFrame(eeg1_0)
    eeg1_1 = pd.DataFrame(eeg1_1)
    eeg1_2 = pd.DataFrame(eeg1_2)

    eeg1_0 = eeg1_0.loc[:, (eeg1_0 != 0).any(axis=0)]
    eeg1_1 = eeg1_1.loc[:, (eeg1_1 != 0).any(axis=0)]
    eeg1_2 = eeg1_2.loc[:, (eeg1_2 != 0).any(axis=0)]
    
    return eeg1_0, eeg1_1, eeg1_2

In [9]:
eeg1_0, eeg1_1, eeg1_2 = get_class_eeg(raw_fnames[0], 'S001R06')
eeg2_0, eeg2_1, eeg2_2 = get_class_eeg(raw_fnames[1], 'S001R10')
eeg3_0, eeg3_1, eeg3_2 = get_class_eeg(raw_fnames[2], 'S001R14')

print(eeg1_0.shape, eeg1_1.shape, eeg1_2.shape)
print(eeg2_0.shape, eeg2_1.shape, eeg2_2.shape)
print(eeg3_0.shape, eeg3_1.shape, eeg3_2.shape)

print(eeg1_0.shape[1] + eeg1_1.shape[1] + eeg1_2.shape[1],
eeg2_0.shape[1] + eeg2_1.shape[1] + eeg2_2.shape[1],
eeg2_0.shape[1] + eeg2_1.shape[1] + eeg2_2.shape[1])

eeg_class0 = pd.concat([eeg1_0, eeg2_0, eeg3_0], axis=1)
eeg_class1 = pd.concat([eeg1_1, eeg2_1, eeg3_1], axis=1)
eeg_class2 = pd.concat([eeg1_2, eeg2_2, eeg3_2], axis=1)

class_0 = eeg_class0.shape[1]
class_1 = eeg_class1.shape[1]
class_2 = eeg_class2.shape[1]

EEG = pd.concat([eeg_class0, eeg_class1, eeg_class2], axis=1)
EEG.columns = list(map(lambda x: str(x), np.arange(0, 60000)))
EEG.shape

Extracting EDF parameters from /Users/alexandr/Documents/Git/MIPT_Masters_work/data/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 265 samples (1.656 sec)

Extracting EDF parameters from /Users/alexandr/Documents/Git/MIPT_Masters_work/data/S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1999

(64, 60000)

In [80]:
def get_score_on_chanells(EEG, channel_names, eeg_class0, eeg_class1, eeg_class2):

    ch_number_0 = [name_ch_dict[i] for i in channel_names]
    EEG = EEG.loc[ch_number_0, :]
    matrix = EEG
    size = matrix.shape
    class_0 = eeg_class0.shape[1]
    class_1 = eeg_class1.shape[1]
    class_2 = eeg_class2.shape[1]


    CHANALS = EEG.shape[0]
    N_COMPONENTS_PCA = int(EEG.shape[0] / 2)
    FREQ = 160

    TIME_SEC = 373.5

    TIME_SIZE_SEC = 2
    STEP_TIME_SEC = 1

    SAMPLE_SIZE = TIME_SIZE_SEC * FREQ
    STEP_TIME = STEP_TIME_SEC * FREQ

    LINSPACE = 0, TIME_SEC, FREQ*TIME_SEC

    from sklearn.decomposition import FastICA

    FastICA = FastICA(n_components=CHANALS).fit(matrix.T)
    ICA = FastICA.transform(matrix.T)

    matrix = ICA.T

    matrix_class1 = matrix[:, 0:class_0]
    matrix_calss2 = matrix[:, class_0:class_0+class_1]
    matrix_calss3 = matrix[:, class_0+class_1:class_0+class_1+class_2]
    #Получаем семплы для каждого класса
    sample_calss1 = get_sample(matrix_class1, sample_size=SAMPLE_SIZE, step=STEP_TIME)
    sample_calss2 = get_sample(matrix_calss2, sample_size=SAMPLE_SIZE, step=STEP_TIME)
    sample_calss3 = get_sample(matrix_calss3, sample_size=SAMPLE_SIZE, step=STEP_TIME)

    i_ = 0
    for i in range(len(sample_calss1)):
        for j in range(sample_calss1[0].shape[0]):
            #print(sample_calss1[i][j].shape[0])
            if sample_calss1[i][j].shape[0] != SAMPLE_SIZE:
                if i_ == 0:
                    i_ = i

    sample_calss1 = sample_calss1[:i_]
    sample_calss2 = sample_calss2[:i_]
    sample_calss3 = sample_calss3[:i_]

    samples_fft = list(fft_for_sample(sample_calss1 + sample_calss2 + sample_calss3, freq=FREQ, lowFreq=7, highFreq=30))

    len_class_1 = len(sample_calss1)
    len_class_2 = len(sample_calss2)
    len_class_3 = len(sample_calss3)

    sample_calss1_fft = samples_fft[:len_class_1]
    sample_calss2_fft = samples_fft[len_class_1:len_class_1 + len_class_2]
    sample_calss3_fft = samples_fft[len_class_1 + len_class_2:len_class_1 + len_class_2 + len_class_3]


    FIRST_N_FFT = len(sample_calss1_fft[0][0])

    #Создание строк для датасета, из матрицы CHANALS*FIRST_N_FFT -> в вектор
    sample_calss1_fft_str = create_strings_for_dataset(sample_calss1_fft)
    sample_calss2_fft_str = create_strings_for_dataset(sample_calss2_fft)
    sample_calss3_fft_str = create_strings_for_dataset(sample_calss3_fft)



    #Создание таблицы объекты-признаки

    #Класс 1
    data_class_1 = pd.DataFrame(data=np.zeros((len_class_1, size[0] * FIRST_N_FFT)))
    data_class_1['label'] = 1

    data_class_1 = np.array(data_class_1)

    for i in(range(len(sample_calss1_fft_str))):
        data_class_1[i, :-1] = sample_calss1_fft_str[i]


    #Класс 2
    data_class_2 = pd.DataFrame(data=np.zeros((len_class_2 - 3, size[0] * FIRST_N_FFT))) #!!!!!!
    data_class_2['label'] = 2

    data_class_2 = np.array(data_class_2)

    for i in (range(len(sample_calss2_fft_str) - 3)): #!!!!!!!!!
        data_class_2[i, :-1] = sample_calss2_fft_str[i]


    #Класс 3
    data_class_3 = pd.DataFrame(data=np.zeros((len_class_3 - 1, size[0] * FIRST_N_FFT))) #####!!!!!!
    data_class_3['label'] = 3

    data_class_3 = np.array(data_class_3)

    for i in (range(len(sample_calss3_fft_str) - 1)): #####!!!!
        data_class_3[i, :-1] = sample_calss3_fft_str[i]




    data = np.vstack([data_class_1, data_class_2, data_class_3])
    data = pd.DataFrame(data)
    print(data.shape)
    data.columns = [*data.columns[:-1], 'label']
    
    from sklearn.model_selection import cross_val_score
    X = data.iloc[:, :-1]
    y = data['label']
    
    from sklearn.decomposition import PCA
    PCA = PCA(n_components=len(channel_names) // 2, random_state=100)

    # Понижаем размерность
    X = PCA.fit_transform(X)
    print(X.shape)
    rf = RandomForestClassifier()
    print(f'score: {np.mean(cross_val_score(rf, X, y, cv=3))} on channels: {channel_names}')

In [81]:
from itertools import combinations
import random
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


channel_names_BEST = ['O2', 'C3', 'C4']
dont_best = [i for i in list(name_ch_dict.keys()) if i not in channel_names_BEST]
dont_best = [i for i in dont_best if i[0] != 'C' and i[0] != 'T' and i[:2] != 'FC']
print('best channels:')
get_score_on_chanells(EEG, channel_names_BEST, eeg_class0, eeg_class1, eeg_class2)
print('other channels:')
for _ in range(20):
    channels = random.sample(dont_best, len(channel_names_BEST))
    get_score_on_chanells(EEG, channels, eeg_class0, eeg_class1, eeg_class2)
    
print()
    
channel_names_BEST = ['O2', 'FC4', 'C3', 'FCz']
channel_names_BEST1 = ['O2','T8','FC4','C3']
dont_best = [i for i in list(name_ch_dict.keys()) if i not in channel_names_BEST]
dont_best = [i for i in dont_best if i[0] != 'C' and i[:2] != 'FC']
print('best channels:')
get_score_on_chanells(EEG, channel_names_BEST, eeg_class0, eeg_class1, eeg_class2)
get_score_on_chanells(EEG, channel_names_BEST1, eeg_class0, eeg_class1, eeg_class2)
print('other channels:')
for _ in range(20):
    channels = random.sample(dont_best, len(channel_names_BEST))
    get_score_on_chanells(EEG, channels, eeg_class0, eeg_class1, eeg_class2)
    
print()
    
# channel_names_BEST = ['P2','O2','C3','C6','Fp1','P5','C4']
# channel_names_BEST1 = ['O2','Fp1','C3','F5','P2','C4']
# dont_best = [i for i in list(name_ch_dict.keys()) if i not in channel_names_BEST]
# dont_best = [i for i in dont_best if i[0] != 'C' and i[:2] != 'FC']
# print('best channels:')
# get_score_on_chanells(EEG, channel_names_BEST, eeg_class0, eeg_class1, eeg_class2)
# get_score_on_chanells(EEG, channel_names_BEST1, eeg_class0, eeg_class1, eeg_class2)
# print('other channels:')
# for _ in range(20):
#     channels = random.sample(dont_best, len(channel_names_BEST))
#     get_score_on_chanells(EEG, channels, eeg_class0, eeg_class1, eeg_class2)
    



best channels:
(370, 220)
(370, 1)
score: 0.43213467301251535 on channels: ['O2', 'C3', 'C4']
other channels:
(370, 220)
(370, 1)
score: 0.3595099594570774 on channels: ['Fp1', 'P6', 'F1']
(370, 220)
(370, 1)
score: 0.4078970562312709 on channels: ['Fpz', 'AFz', 'AF3']
(370, 220)
(370, 1)
score: 0.33505200070509433 on channels: ['P4', 'O1', 'P5']
(370, 220)
(370, 1)
score: 0.3866120218579235 on channels: ['AF7', 'AF3', 'F7']
(370, 220)
(370, 1)
score: 0.4082936717786004 on channels: ['P4', 'PO7', 'FT7']
(370, 220)
(370, 1)
score: 0.4569011105235325 on channels: ['F6', 'Iz', 'Pz']
(370, 220)
(370, 1)
score: 0.3620218579234973 on channels: ['P6', 'Fpz', 'AFz']
(370, 220)
(370, 1)
score: 0.394632469592808 on channels: ['P7', 'Fz', 'P5']
(370, 220)
(370, 1)
score: 0.39432399083377406 on channels: ['PO3', 'PO4', 'POz']
(370, 220)
(370, 1)
score: 0.3811034725894588 on channels: ['F4', 'P3', 'Fpz']
(370, 220)
(370, 1)
score: 0.47563017803631236 on channels: ['P6', 'AF8', 'F7']
(370, 220)
(370